# **Обучение модели**

Загрузка нужных библиотек для работы

In [ ]:
!pip3 install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.5/98.5 MB 6.4 MB/s eta 0:00:00


In [ ]:
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 6.5 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import ElasticNet
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostRegressor
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform, randint
import optuna

**Загрузка датасета**

In [53]:
df_realty = pd.read_csv('realty_data_model.csv')
display(df_realty.head())

,baths,fireplace,sqft,target,pool,Heating,Cooling,Parking,school_rating _mean,school_dist_min,...,Type_mobile_home,Type_multi_family,Type_other,Type_ranch,Type_single_family,Type_townhouse,state_label,year_ord,city_label,zip_label
0,1.098613,True,7.972466,12.943237,False,True,False,False,1.648659,0.993252,...,0.0,0.0,0.0,0.0,1.0,0.0,20,215,1578,815
1,1.098613,False,7.574045,12.644328,False,False,False,False,1.386295,0.009951,...,0.0,0.0,0.0,0.0,1.0,0.0,35,215,1588,4243
2,0.693148,True,8.006368,14.878496,True,True,True,True,1.902108,0.173954,...,0.0,0.0,0.0,0.0,1.0,0.0,2,157,959,3382
3,2.079442,True,8.772920,14.688894,False,True,True,True,1.722767,-2.302575,...,0.0,0.0,0.0,0.0,1.0,0.0,31,202,398,2436
4,0.693148,False,-13.815511,8.517193,False,False,False,False,1.547563,1.108563,...,0.0,0.0,0.0,0.0,0.0,0.0,6,219,1244,1103


In [54]:
df_realty.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 367668 entries, 0 to 367667
Data columns (total 73 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   baths                    367668 non-null  float64
 1   fireplace                367668 non-null  bool   
 2   sqft                     367668 non-null  float64
 3   target                   367668 non-null  float64
 4   pool                     367668 non-null  bool   
 5   Heating                  367668 non-null  bool   
 6   Cooling                  367668 non-null  bool   
 7   Parking                  367668 non-null  bool   
 8   school_rating _mean      367668 non-null  float64
 9   school_dist_min          367668 non-null  float64
 10  status_Active            367668 non-null  float64
 11  status_Auction           367668 non-null  float64
 12  status_Backup Contract   367668 non-null  float64
 13  status_Coming Soon       367668 non-null  float64
 14  stat

In [55]:
y = df_realty.target.values
X = df_realty.drop(['target'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=True, random_state=42)

print("X_train",X_train.shape)
print("y_train",y_train.shape)

print("X_test",X_test.shape)
print("y_test",y_test.shape)

X_train (257367, 72)
y_train (257367,)
X_test (110301, 72)
y_test (110301,)


# Модель 1 **LinearRegression**

In [56]:
lr_model = LinearRegression()

lr_model.fit(X_train, y_train)

# делаем предсказания
y_train_pred_lr = lr_model.predict(X_train)
y_test_pred_lr = lr_model.predict(X_test)

# вычисляем метрики
mse_train = metrics.mean_squared_error(y_train, y_train_pred_lr)
mse_test = metrics.mean_squared_error(y_test, y_test_pred_lr)
mae_train = metrics.mean_absolute_error(y_train, y_train_pred_lr)
mae_test = metrics.mean_absolute_error(y_test, y_test_pred_lr)
r2_train = metrics.r2_score(y_train, y_train_pred_lr)
r2_test = metrics.r2_score(y_test, y_test_pred_lr)
rmse = np.sqrt(mse_test)


print(f"Train MSE: {mse_train:.2f}")
print(f"Test MSE: {mse_test:.2f}")
print(f"Train MAE: {mae_train:.2f}")
print(f"Test MAE: {mae_test:.2f}")
print(f"Train R2: {r2_train:.2f}")
print(f"Test R2: {r2_test:.2f}")
print(f"Test RMSE: {rmse:.2f}")

Train MSE: 1.11
Test MSE: 1.11
Train MAE: 0.76
Test MAE: 0.76
Train R2: 0.17
Test R2: 0.17
Test RMSE: 1.05


**MSE** (Mean Squared Error) равен 1.11, что указывает на высокую ошибку модели.

**MAE** (Mean Absolute Error) равен 0.76, что указывает на среднюю абсолютную ошибку модели.

**R2** (R-squared) равен 0.17, что указывает на низкую способность модели объяснять дисперсию зависимой переменной.


Исходя из этих метрик, можно сказать, что модель линейной регрессии имеет высокую среднеквадратичную ошибку и среднюю абсолютную ошибку, а также невысокий коэффициент детерминации R2, что может указывать на не очень хорошее качество модели.

# Модель 2 **RandomForestRegressor**

In [57]:
rand_forest_regressor = RandomForestRegressor(random_state=42)

rand_forest_regressor.fit(X_train, y_train)

y_train_pred_rfr = rand_forest_regressor.predict(X_train)
y_test_pred_rfr = rand_forest_regressor.predict(X_test)

# вычисляем метрики
mse_train = mean_squared_error(y_train, y_train_pred_rfr)
mse_test = mean_squared_error(y_test, y_test_pred_rfr)
mae_train = mean_absolute_error(y_train, y_train_pred_rfr)
mae_test = mean_absolute_error(y_test, y_test_pred_rfr)
r2_train = r2_score(y_train, y_train_pred_rfr)
r2_test = r2_score(y_test, y_test_pred_rfr)
rmse = np.sqrt(mse_test)


print(f"Train MSE: {mse_train:.2f}")
print(f"Test MSE: {mse_test:.2f}")
print(f"Train MAE: {mae_train:.2f}")
print(f"Test MAE: {mae_test:.2f}")
print(f"Train R2: {r2_train:.2f}")
print(f"Test R2: {r2_test:.2f}")
print(f"Test RMSE: {rmse:.2f}")

Train MSE: 0.04
Test MSE: 0.26
Train MAE: 0.11
Test MAE: 0.29
Train R2: 0.97
Test R2: 0.80
Test RMSE: 0.51


**MSE** (Mean Squared Error) низкий как для обучающей (0.04), так и для тестовой (0.26) выборок, что указывает на хорошую обобщающую способность модели.

**MAE** (Mean Absolute Error) также низкий для обучающей (0.11) и тестовой (0.29) выборок.

**R2** (R-squared) высокий для обучающей (0.97) и тестовой (0.80) выборок, что говорит о хорошей способности модели объяснять дисперсию зависимой переменной и её применимости к новым данным.

**RMSE** (Root Mean Squared Error) для тестовой выборки составляет 0.51, что указывает на хорошую предсказательную способность модели.


В целом, модель RandomForestRegressor демонстрирует отличные результаты, с низкой средней квадратичной и абсолютной ошибкой, высоким коэффициентом детерминации R2 и низким значением RMSE, что подтверждает её хорошую обобщающую способность и применимость к новым данным.

In [67]:
# Сохраняем модель в файл с помощью pickle.dump
with open('rand_forest_regressor.pkl', 'wb') as file:
    pickle.dump(rand_forest_regressor, file)

# Модель 3 **ElasticNetCV**

In [58]:
el_netcv_model = ElasticNetCV(cv=5, random_state=42,)
el_netcv_model.fit(X_train, y_train)

y_train_pred_el = el_netcv_model.predict(X_train)
y_test_pred_el = el_netcv_model.predict(X_test)

# вычисляем метрики
mse_train = mean_squared_error(y_train, y_train_pred_el)
mse_test = mean_squared_error(y_test, y_test_pred_el)
mae_train = mean_absolute_error(y_train, y_train_pred_el)
mae_test = mean_absolute_error(y_test, y_test_pred_el)
r2_train = r2_score(y_train, y_train_pred_el)
r2_test = r2_score(y_test, y_test_pred_el)
rmse = np.sqrt(mse_test)


print(f"Train MSE: {mse_train:.2f}")
print(f"Test MSE: {mse_test:.2f}")
print(f"Train MAE: {mae_train:.2f}")
print(f"Test MAE: {mae_test:.2f}")
print(f"Train R2: {r2_train:.2f}")
print(f"Test R2: {r2_test:.2f}")
print(f"Test RMSE: {rmse:.2f}")

Train MSE: 1.18
Test MSE: 1.17
Train MAE: 0.78
Test MAE: 0.78
Train R2: 0.13
Test R2: 0.13
Test RMSE: 1.08







Модель ElasticNetCV имеет высокую среднеквадратичную ошибку (MSE) и среднюю абсолютную ошибку (MAE) на обучающей и тестовой выборках, низкий коэффициент детерминации (R2) и высокое значение RMSE на тестовой выборке. Это указывает на плохую предсказательную способность модели и ее неспособность обобщать данные и работать с новыми данными.

# Модель 3 **CatBoostRegressor  RandomizedSearchCV**

Загрузка датасета для модели CatBoost

In [45]:
realty_data_full = pd.read_csv('data_catmodel.csv')

In [46]:
cat_features = ['status','city','zipcode','state','Type','Year built']
num_features = ['baths', 'sqft', 'target', 'school_rating _mean', 'school_dist_min']

In [47]:
bool_features = ['pool','Heating','Cooling','Parking','fireplace']

In [48]:
def log_data(data):
    cat_data = data.copy()
    # переведем признак зип кода в категориальный
    cat_data['zipcode'] = cat_data['zipcode'].astype(str)
    # переведем признак год в категориальный
    cat_data['Year built'] = cat_data['Year built'].astype(str)

    for column in num_features:
        cat_data[column] = cat_data[column].apply(lambda x: abs(x))
        constant = 1e-6
        cat_data[column] = np.log(cat_data[column] + constant)
    return cat_data

In [49]:
data_log = log_data(realty_data_full)
data_log.sample(5)

,status,baths,fireplace,city,sqft,zipcode,state,target,Type,pool,Year built,Heating,Cooling,Parking,school_rating _mean,school_dist_min
828,Active,6.931477e-01,False,Miami,6.857514,33125,FL,12.375815,condos,False,2007,True,True,True,0.693148,-1.171180
186745,other,1.945910e+00,False,Pinecrest,8.858937,33156,FL,14.508158,single_family,True,2008,True,True,True,1.791760,-0.916288
333686,other,9.999995e-07,False,Gulfport,6.815640,33707,FL,11.651817,single_family,False,1952,True,True,True,0.993252,-0.916288
305102,Active,6.931477e-01,False,Tampa,-13.815511,33605,FL,11.002100,land,False,no date,False,False,False,0.993252,-0.478034
190197,Pending,6.931477e-01,False,Fairview Park,7.352441,44126,OH,11.651817,single_family,False,1930,True,False,False,1.757858,-1.609433


In [50]:
data_log.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 368694 entries, 0 to 368693
Data columns (total 16 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   status               368694 non-null  object 
 1   baths                368694 non-null  float64
 2   fireplace            368694 non-null  bool   
 3   city                 368694 non-null  object 
 4   sqft                 368694 non-null  float64
 5   zipcode              368694 non-null  object 
 6   state                368694 non-null  object 
 7   target               368694 non-null  float64
 8   Type                 368694 non-null  object 
 9   pool                 368694 non-null  bool   
 10  Year built           368694 non-null  object 
 11  Heating              368694 non-null  bool   
 12  Cooling              368694 non-null  bool   
 13  Parking              368694 non-null  bool   
 14  school_rating _mean  368694 non-null  float64
 15  school_dist_min  

Предположение,что если закодировать булевые признаки,то модель будет лучше обучаться,снизится RMSE (Root Mean Squared Error)

In [51]:
for feature in bool_features:
    data_log[feature] = data_log[feature].map({True: 'Yes', False: 'No'})

# Проверка результата
print(data_log[bool_features].head())

  pool Heating Cooling Parking fireplace
0   No     Yes      No      No       Yes
1   No      No      No      No        No
2  Yes     Yes     Yes     Yes       Yes
3   No     Yes     Yes     Yes       Yes
4   No      No      No      No        No


In [52]:
y = data_log.target.values
X = data_log.drop(['target'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=42)

In [35]:
param_dist = {
    'iterations': randint(100, 200),
    'learning_rate': uniform(0.01, 0.1 - 0.01),
    'depth': randint(4, 8),
    'l2_leaf_reg': randint(1, 5),
}

cb_model = CatBoostRegressor(random_seed=42, silent=True)
random_search = RandomizedSearchCV(estimator=cb_model, param_distributions=param_dist, n_iter=50, cv=5, scoring='neg_mean_squared_error', verbose=2, n_jobs=-1)
random_search.fit(X_train, y_train, cat_features=cat_features)
best_params = random_search.best_params_
best_cb_model = CatBoostRegressor(iterations=best_params['iterations'], learning_rate=best_params['learning_rate'], depth=best_params['depth'], l2_leaf_reg=best_params['l2_leaf_reg'], random_seed=42, silent=True)
best_cb_model.fit(X_train, y_train, cat_features=cat_features)

y_train_pred = best_cb_model.predict(X_train)
y_test_pred = best_cb_model.predict(X_test)

# вычисляем метрики
mse_train = mean_squared_error(y_train, y_train_pred)
mse_test = mean_squared_error(y_test, y_test_pred)
mae_train = mean_absolute_error(y_train, y_train_pred)
mae_test = mean_absolute_error(y_test, y_test_pred)
r2_train = r2_score(y_train, y_train_pred)
r2_test = r2_score(y_test, y_test_pred)

print(f"Train MSE: {mse_train:.2f}")
print(f"Test MSE: {mse_test:.2f}")
print(f"Train MAE: {mae_train:.2f}")
print(f"Test MAE: {mae_test:.2f}")
print(f"Train R2: {r2_train:.2f}")
print(f"Test R2: {r2_test:.2f}")

Fitting 5 folds for each of 50 candidates, totalling 250 fits


/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Train MSE: 0.30
Test MSE: 0.30
Train MAE: 0.35
Test MAE: 0.35
Train R2: 0.78
Test R2: 0.78


1. MSE (Mean Squared Error):
   - Значение MSE для обучающей и тестовой выборок одинаково и составляет 0.30. Это означает, что среднеквадратичная ошибка модели на обучающей и тестовой выборках низкая.

2. MAE (Mean Absolute Error):
   - Значение MAE для обучающей и тестовой выборок также одинаково и составляет 0.35. Это говорит о том, что средняя абсолютная ошибка модели на обучающей и тестовой выборках низкая.

3. R2 (R-squared):
   - Значение R2 для обучающей и тестовой выборок равно 0.78. Это высокое значение коэффициента детерминации, что указывает на хорошее качество модели в объяснении дисперсии зависимой переменной как на обучающей, так и на тестовой выборке.


Модель CatBoostRegressor RandomizedSearchCV хорошо работает как на обучающей, так и на тестовой выборке, и способна хорошо обобщать данные и делать точные прогнозы.

In [36]:
import pickle

# Сохраняем модель в файл с помощью pickle.dump
with open('best_cb_model.pkl', 'wb') as file:
    pickle.dump(best_cb_model, file)

Модель 3 **CatBoostRegressor Optuna**

In [ ]:
train_df = data_log[data_log['target'].notnull()]
X = train_df.drop('target', axis=1)
y = train_df['target']

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X,y,test_size = 0.2, random_state=42)

In [ ]:
print(X_train.shape)
print(X_val.shape)
print(y_train.shape)
print(y_val.shape)

(294955, 15)
(73739, 15)
(294955,)
(73739,)


In [ ]:
def objective(trial):
    params = {
        "iterations": 200,
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.1, log=True),
        "depth": trial.suggest_int("depth", 1, 10),
        "subsample": trial.suggest_float("subsample", 0.05, 1.0),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.05, 1.0),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 1, 100),
    }

    cbr_model = CatBoostRegressor(**params, random_state=42)

    cbr_model.fit(X_train, y_train,
                  eval_set=[(X_val, y_val)],
                  cat_features=cat_features,
                  early_stopping_rounds=10, # Ранняя остановка
                  verbose=False) # Не выводить лишнюю информацию

    rmse = np.sqrt(mean_squared_error(y_val, cbr_model.predict(X_val)))
    return rmse

In [ ]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)

[I 2024-04-08 08:31:23,729] A new study created in memory with name: no-name-0fcbc259-8f14-49de-8766-4327c00d7052
[I 2024-04-08 08:32:01,486] Trial 0 finished with value: 0.5970403152313429 and parameters: {'learning_rate': 0.04044577348112126, 'depth': 4, 'subsample': 0.31212516062951323, 'colsample_bylevel': 0.5732753067354753, 'min_data_in_leaf': 40}. Best is trial 0 with value: 0.5970403152313429.
[I 2024-04-08 08:34:32,974] Trial 1 finished with value: 0.8827402828732004 and parameters: {'learning_rate': 0.00258963131849818, 'depth': 10, 'subsample': 0.98674855081927, 'colsample_bylevel': 0.673649060090247, 'min_data_in_leaf': 72}. Best is trial 0 with value: 0.5970403152313429.
[I 2024-04-08 08:35:44,591] Trial 2 finished with value: 0.9100053787758778 and parameters: {'learning_rate': 0.0026947052742356675, 'depth': 6, 'subsample': 0.5825908205841137, 'colsample_bylevel': 0.7661998058454703, 'min_data_in_leaf': 94}. Best is trial 0 with value: 0.5970403152313429.
[I 2024-04-08 0

In [ ]:
print(f'Best parameters: {study.best_params}')
print(f'Best RMSE: {study.best_value}')

Best parameters: {'learning_rate': 0.09769892190141977, 'depth': 10, 'subsample': 0.5634178693115097, 'colsample_bylevel': 0.8193891543729548, 'min_data_in_leaf': 5}
Best RMSE: 0.4991420837311278


In [ ]:
cbr_model_final = CatBoostRegressor(**study.best_params, random_state=42)
cbr_model_final.fit(X,y,cat_features=cat_features,verbose=False)

Mодель CatBoostRegressor с оптимизацией параметров Optuna достигла хорошего значения RMSE 0.499 и хорошо предсказывает зависимую переменную на тестовой выборке,показывает хорошие результаты и способна делать точные прогнозы на основе предоставленных данных.

Анализ важности признаков (упорядочены по убыванию их важности)

In [ ]:
feature_importances = pd.DataFrame({'feature_importance': cbr_model_final.feature_importances_, 'feature_names': X_val.columns})
feature_importances.sort_values(by=['feature_importance'], ascending=False, inplace=True)
display(feature_importances)

,feature_importance,feature_names
4,22.418241,sqft
5,16.607768,zipcode
3,14.931015,city
7,11.729381,Type
6,9.544497,state
9,5.781319,Year built
13,4.395059,school_rating _mean
0,3.944622,status
1,3.607946,baths
14,2.781086,school_dist_min


Из полученной таблицы мы видим,что более важными признаками для модели являются sqft, zipcode, city, Type.

А признаки,о которых мы предполагали,что они будут иметь значение Cooling, fireplace, Heating, Parking,pool (булевые значения) не информативны.

In [ ]:
import pickle

# Сохраняем модель в файл с помощью pickle.dump
with open('cbr_model_final.pkl', 'wb') as file:
    pickle.dump(cbr_model_final, file)

Модель 3.1 **CatBoostRegressor Optuna** с доработками

Предположение,что датасет без вышеперечисленных булевых признаков будет лучше работать на новых данных и  давать более точные прогнозы

In [37]:
train_df_optuna = data_log[data_log['target'].notnull()]
X = train_df_optuna.drop('target', axis=1)
y = train_df_optuna['target']

In [38]:
X_train, X_val, y_train, y_val = train_test_split(X,y,test_size = 0.2, random_state=42)

In [39]:
categorise_features = ['status','city','zipcode','state','Type','Year built']

In [40]:
def objective(trial):
    params = {
        "iterations": 200,
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.1, log=True),
        "depth": trial.suggest_int("depth", 1, 10),
        "subsample": trial.suggest_float("subsample", 0.05, 1.0),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.05, 1.0),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 1, 100),
    }

    cbr_optuna_model = CatBoostRegressor(**params, random_state=42)

    cbr_optuna_model.fit(X_train, y_train,
                  eval_set=[(X_val, y_val)],
                  cat_features=categorise_features,
                  early_stopping_rounds=20, # Ранняя остановка для предотвращения переобучения
                  verbose=False) # Не выводить лишнюю информацию

    rmse = np.sqrt(mean_squared_error(y_val, cbr_optuna_model.predict(X_val)))

    return rmse

In [42]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)

[I 2024-04-08 15:23:11,538] A new study created in memory with name: no-name-c4f2f220-9773-4aee-ac19-8561c24fb02a
[I 2024-04-08 15:23:44,499] Trial 0 finished with value: 0.5611718089197691 and parameters: {'learning_rate': 0.06073503226386861, 'depth': 5, 'subsample': 0.22756301685797609, 'colsample_bylevel': 0.5261280337768538, 'min_data_in_leaf': 59}. Best is trial 0 with value: 0.5611718089197691.
[I 2024-04-08 15:24:28,043] Trial 1 finished with value: 0.9905165905281589 and parameters: {'learning_rate': 0.0015358597702989022, 'depth': 9, 'subsample': 0.274087919100301, 'colsample_bylevel': 0.20933440939587206, 'min_data_in_leaf': 3}. Best is trial 0 with value: 0.5611718089197691.
[I 2024-04-08 15:25:41,037] Trial 2 finished with value: 0.5205680133584656 and parameters: {'learning_rate': 0.09928030719138353, 'depth': 7, 'subsample': 0.8515608022272978, 'colsample_bylevel': 0.5315833959726509, 'min_data_in_leaf': 90}. Best is trial 2 with value: 0.5205680133584656.
[I 2024-04-08 

In [43]:
print(f'Best parameters: {study.best_params}')
print(f'Best RMSE: {study.best_value}')


Best parameters: {'learning_rate': 0.07837889196135095, 'depth': 10, 'subsample': 0.9990453722823485, 'colsample_bylevel': 0.5728958153879138, 'min_data_in_leaf': 86}
Best RMSE: 0.5045294246200022


In [44]:
cbr_model_optuna_2 = CatBoostRegressor(**study.best_params, random_state=42)
cbr_model_optuna_2.fit(X,y,cat_features=categorise_features,verbose=False)

Данная доработанная модель имеет результат RMSE,который незначительно хуже,чем в исходной модели

**Посмотрим на все полученные метрики по моделям**

In [66]:
data_models = {'Metric': ['Train MSE', 'Test MSE', 'Train MAE', 'Test MAE', 'Train R2', 'Test R2','Test RMSE'],
        'LinearRegression': [1.11, 1.11, 0.76, 0.76, 0.17, 0.17, 1.05],
        'RandomForestRegressor': [0.04, 0.26, 0.11, 0.29, 0.97, 0.80, 0.51],
        'ElasticNetCV': [1.18, 1.17, 0.78, 0.78, 0.13, 0.13, 1.08],
        'CatBoostRegressor RandomizedSearchCV': [0.30, 0.30, 0.35, 0.35, 0.78, 0.78, '-'],
        'CatBoostRegressor Optuna':['-', '-', '-', '-', '-', '-', 0.49],
        'CatBoostRegressor Optuna2':['-', '-', '-', '-', '-', '-', 0.50]}

data_models_metric = pd.DataFrame(data_models)
display(data_models_metric)

,Metric,LinearRegression,RandomForestRegressor,ElasticNetCV,CatBoostRegressor RandomizedSearchCV,CatBoostRegressor Optuna,CatBoostRegressor Optuna2
0,Train MSE,1.11,0.04,1.18,0.3,-,-
1,Test MSE,1.11,0.26,1.17,0.3,-,-
2,Train MAE,0.76,0.11,0.78,0.35,-,-
3,Test MAE,0.76,0.29,0.78,0.35,-,-
4,Train R2,0.17,0.97,0.13,0.78,-,-
5,Test R2,0.17,0.80,0.13,0.78,-,-
6,Test RMSE,1.05,0.51,1.08,-,0.49,0.5


Самые лучшие параметры среди 5 моделей  в совокупности имеет простая модель RandomForestRegressor. Она имеет хорошую обобщающую способность и воспроизводимость,применимость к новым данным.